In [4]:
import torch
print("¿GPU disponible?:", torch.cuda.is_available())
print("Nombre de la GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No hay GPU")


¿GPU disponible?: True
Nombre de la GPU: NVIDIA GeForce RTX 4070 Laptop GPU


In [5]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Definir transformaciones para aumentar los datos
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Tamaño estándar de ResNet50
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalización
])

# Cargar datos
# Definir directorios
base_dir = r"C:\PROYECTO_FINAL\images\melanoma_cancer_dataset"  # Cambia esto a la ubicación de tus datos
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')


train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Crear DataLoaders para entrenamiento y prueba
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Clases
print("Clases:", train_dataset.classes)


Clases: ['benign', 'malignant']


In [6]:
import torch.nn as nn
import torchvision.models as models

# Cargar modelo preentrenado
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model = model.to(device)

# Modificar la última capa para 2 clases (benigno vs maligno)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)  # 1 neurona para clasificación binaria
model.fc = model.fc.to(device)

print(model)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\jakif/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 42.4MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
import torch.optim as optim

# Definir función de pérdida y optimizador
criterion = nn.BCEWithLogitsLoss()  # Usa sigmoid internamente
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [8]:
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()  # Modo entrenamiento
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device, dtype=torch.float32)

            optimizer.zero_grad()
            outputs = model(images).squeeze()  # Remover dimensiones extra
            loss = criterion(outputs, labels)  # Calcular pérdida
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.round(torch.sigmoid(outputs))  # Convertir logits en 0 o 1
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        accuracy = 100 * correct / total
        print(f"Época {epoch+1}/{epochs}, Pérdida: {running_loss/len(train_loader):.4f}, Precisión: {accuracy:.2f}%")

    print("Entrenamiento completado")

# Entrenar el modelo
train_model(model, train_loader, criterion, optimizer, epochs=10)


Época 1/10, Pérdida: 0.2348, Precisión: 91.16%
Época 2/10, Pérdida: 0.1591, Precisión: 93.93%
Época 3/10, Pérdida: 0.1405, Precisión: 94.34%
Época 4/10, Pérdida: 0.1177, Precisión: 95.45%
Época 5/10, Pérdida: 0.1035, Precisión: 95.84%
Época 6/10, Pérdida: 0.0938, Precisión: 96.25%
Época 7/10, Pérdida: 0.0834, Precisión: 96.87%
Época 8/10, Pérdida: 0.0644, Precisión: 97.32%
Época 9/10, Pérdida: 0.0635, Precisión: 97.57%
Época 10/10, Pérdida: 0.0500, Precisión: 98.14%
Entrenamiento completado


In [9]:
torch.save(model.state_dict(), 'melanoma_model_1_torch.pth')

In [16]:
import cv2
from PIL import Image

def predict_image(image_path, model):
    model.eval()
    img = Image.open(image_path)
    img = transform(img).unsqueeze(0).to(device)  # Aplicar las mismas transformaciones

    with torch.no_grad():
        output = model(img).squeeze()
        prediction = torch.sigmoid(output).item()
    
    if prediction > 0.5:
        print("🔴 Melanoma maligno detectado")
    else:
        print("🟢 Melanoma benigno detectado")

# Prueba con una imagen nueva

predict_image('C:\\PROYECTO_FINAL\\images\\ISIC-images(bcn20000)\\ISIC_0053453.jpg', model)#benigno
predict_image('C:\\PROYECTO_FINAL\\images\\ISIC-images(bcn20000)\\ISIC_0053454.jpg', model)#malignant
predict_image('C:\\PROYECTO_FINAL\\images\\ISIC-images(bcn20000)\\ISIC_0053458.jpg', model)#benigno
predict_image('C:\\PROYECTO_FINAL\\images\\ISIC-images(bcn20000)\\ISIC_0053460.jpg', model)#malignant
predict_image('C:\\PROYECTO_FINAL\\images\\melanoma_cancer_dataset\\test\\malignant\\melanoma_10127.jpg', model)
predict_image('C:\\PROYECTO_FINAL\\images\\melanoma_cancer_dataset\\test\\benign\\melanoma_9607.jpg', model)
predict_image('C:\\PROYECTO_FINAL\\images\\melanoma_cancer_dataset\\test\\malignant\\melanoma_10105.jpg', model)
predict_image('C:\\PROYECTO_FINAL\\images\\melanoma_cancer_dataset\\test\\benign\\melanoma_9607.jpg', model)
predict_image('C:\\PROYECTO_FINAL\\images\\melanoma_cancer_dataset\\test\\malignant\\melanoma_10127.jpg', model)
predict_image('C:\\PROYECTO_FINAL\\images\\melanoma_cancer_dataset\\test\\benign\\melanoma_9607.jpg', model)
predict_image('C:\\PROYECTO_FINAL\\images\\melanoma_cancer_dataset\\test\\malignant\\melanoma_10105.jpg', model)

🔴 Melanoma maligno detectado
🔴 Melanoma maligno detectado
🔴 Melanoma maligno detectado
🔴 Melanoma maligno detectado
🔴 Melanoma maligno detectado
🟢 Melanoma benigno detectado
🟢 Melanoma benigno detectado
🟢 Melanoma benigno detectado
🔴 Melanoma maligno detectado
🟢 Melanoma benigno detectado
🔴 Melanoma maligno detectado
